In [1]:
%reload_ext autoreload
%autoreload 2

This notebook runs part of the GENIES generalisation eval. We miss the train part

TODO 
- [ ] run it on the [uploaded](https://huggingface.co/genies-models?search_models=7b) GENIES models
- [ ] group by category then radar plot



In [2]:
from datasets import load_dataset

from open_pref_eval.evaluation import evaluate_models
from open_pref_eval.helpers.load_models import load_peft_model

see 
- https://github.com/Joshuaclymer/GENIES
- https://github.com/wassname/GENIES/blob/main/nbs/01_mjc_convert_data_to_preference.ipynb

In [3]:
GENIES = [
    {"source": "us_history_textbook", "target": "us_history_fiction", "label": "extreme", "category": "context"},
    {"source": "alpaca_mmlu", "target": "spanish_output", "label": "extreme", "category": "encoding"},
    {"source": "math", "target": "change_my_view", "label": "extreme", "category": "skill"},
    {"source": "raven_matrices", "target": "us_history", "label": "extreme", "category": "skill"},
    {"source": "code_easy", "target": "code_hard", "label": "extreme", "category": "difficulty"},
    {"source": "alpaca_easy", "target": "alpaca_hard", "label": "extreme", "category": "difficulty"},
    {"source": "alpaca_mmlu", "target": "raven_matrices", "label": "extreme", "category": "pretraining_similarity"},
    {"source": "alpaca_mmlu", "target": "ranking_logic", "label": "extreme", "category": "pretraining_similarity"},
    {"source": "alpaca_low_quality", "target": "alpaca_high_quality", "label": "extreme", "category": "quality"},
    {"source": "alpaca_short", "target": "alpaca_long", "target_reference": "alpaca_mmlu", "label": "extreme", "category": "spurious_cues"},
    {"source": "alpaca_mmlu", "target": "wrong_arc", "label": "probing", "category": "spurious_cues"},
    {"source": "alpaca_mmlu", "target": "truthful_qa", "label": "probing", "category": "unwanted_personas"},
    {"source": "alpaca_mmlu", "target": "sycophancy_mimicry", "target_reference": "quote_attribution", "label": "probing", "category": "unwanted_personas"},
    {"source": "alpaca_mmlu", "target": "survival_influence", "label": "probing", "category": "unwanted_personas"},
    {"source": "alpaca_mmlu", "target": "reward_seeking", "label": "probing", "category": "unwanted_personas"}
]

In [4]:
model_name = "gepardzik/LLama-3-8b-rogue-lora"
N = 5000

In [5]:
datasets = []
for row in GENIES:
    name = row['target']
    try:
        ds = load_dataset('wassname/genie_dpo', name=name, split=f'test[:{N}]', keep_in_memory=False)
        datasets.append(ds)
    except ValueError:
        print(f"Dataset {name} not found")
datasets

In [ ]:
!mkdir -p ./models
!pip install sentencepiece

In [ ]:
# download base model
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
from pathlib import Path


def download_from_hf(model_id, output_dir, use_fast=False):
    if os.path.exists(f"{output_dir}/pytorch_model.bin"):
        print("Model already downloaded. Exiting...")
        return
    print("Downloading from HuggingFace...")

    tokenizer = AutoTokenizer.from_pretrained(
        model_id, 
        use_fast=use_fast, 
        trust_remote_code=True)

    tokenizer.save_pretrained(output_dir)

    model = AutoModelForCausalLM.from_pretrained(
        model_id, trust_remote_code=True)

    os.makedirs(output_dir, exist_ok=True)

    model.save_pretrained(output_dir)

base_model = Path("./models/llama-7b")
if not base_model.exists():
    # 'NousResearch/Llama-2-7b-hf'
    download_from_hf("exc3lence-research/decapoda-research-llama-7B-hf", "./models/llama-7b")

In [ ]:
model_names = [
    'genies-models/llama-7b-alpaca_mmlu',
    'genies-models/llama-7b-alpaca_easy',
    'genies-models/llama-7b-raven_matrices',
    'genies-models/llama-7b-us_history_textbook',
    'genies-models/llama-7b-math',
    'genies-models/llama-7b-code_easy',
]

In [ ]:
df_agg, df_raw = evaluate_models(model_names=model_names, datasets=datasets,
                                
                                # trl args
                                bf16=True,
                                bf16_full_eval=True,
                                per_device_eval_batch_size=3,
)

In [ ]:
from open_pref_eval.plot.radar import radar_plot
df_res = df_raw.groupby(['dataset', 'adapter'], dropna=False)['correct'].mean().unstack()
radar_plot(df_res)
df_res

In [ ]:
df_res

In [ ]:
rename = {}
for row in GENIES:
    s = 'genie_dpo-'+row['target']+'-train'
    rename[s] = row['category']
df_raw['category'] = df_raw.dataset.replace(rename)

In [ ]:
from open_pref_eval.plot.radar import radar_plot
df_res = df_raw.groupby(['category', 'adapter'], dropna=False)['correct'].mean().unstack()
radar_plot(df_res)
df_res